In [1]:
import io, os, sys, types

In [2]:
import nbformat

from IPython import get_ipython
from IPython.core.interactiveshell import InteractiveShell

In [3]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [4]:
class NotebookLoader(object):
    """Module Loader for IPython Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing notebook from %s" % path)
                                       
        # load the notebook object
        nb = nbformat.read(path, as_version=4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [5]:
class NotebookFinder(object):
    """Module finder that locates IPython Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [6]:
sys.meta_path.append(NotebookFinder())

In [7]:
ls .

2.2.tf_single_layer_regression.ipynb
2.3.tf_single_layer_binary.ipynb
2.4.tf_single_layer_select.ipynb
2.5.notf_single_layer_regression.ipynb
2.6.notf_single_layer_binary.ipynb
2.7.notf_single_layer_select.ipynb
3.1.1.tf_multi_layer_regression.ipynb
3.1.2.tf_multi_layer_h3_regression.ipynb
3.1.3.tf_multi_layer_hn_regression.ipynb
3.1.4.tf_multiple_layer_binary.ipynb
3.1.5.tf_multiple_layer_h3_binary.ipynb
3.1.6.tf_multiple_layer_hn_binary.ipynb
3.1.7.tf_multiple_layer_select.ipynb
3.1.8.tf_multiple_layer_h3_select.ipynb
3.1.9.tf_multiple_layer_hn_select.ipynb
3.2.1.notf_multiple_layer_regression.ipynb
3.2.2.notf_multiple_layer_h3_regression.ipynb
3.2.3.notf_multiple_layer_hn_regression.ipynb
3.2.4.notf_multiple_layer_binary.ipynb
3.2.5.notf_multiple_layer_h3_binary.ipynb
3.2.6.notf_multiple_layer_hn_binary.ipynb
3.2.7.notf_multiple_layer_select.ipynb
3.2.8.notf_multiple_layer_h3_select.ipynb
3.2.9.notf_multiple_layer_hn_select.ipynb
3.3.tf_multiple_layer_model.ipynb
3.4.notf_multiple_l

In [8]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

from IPython.display import display, HTML

formatter = HtmlFormatter()
lexer = PythonLexer()

# publish the CSS for pygments highlighting
display(HTML("""
<style type='text/css'>
%s
</style>
""" % formatter.get_style_defs()
))

In [9]:
def show_notebook(fname):
    """display a short summary of the cells of a notebook"""
    nb = nbformat.read(fname, as_version=4)
    html = []
    for cell in nb.cells:
        html.append("<h4>%s cell</h4>" % cell.cell_type)
        if cell.cell_type == 'code':
            html.append(highlight(cell.source, lexer, formatter))
        else:
            html.append("<pre>%s</pre>" % cell.source)
    display(HTML('\n'.join(html)))

show_notebook(os.path.join(".", "3.4.notf_multiple_layer_model.ipynb"))